In [ ]:
import os
import pandas as pd
import openai
from openai import OpenAI, beta
import sys
import json
import pprint
from tqdm import tqdm

sys.path.append(os.path.abspath(".."))
from src import config

from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
  api_key=os.getenv('OPENAI_API_KEY')
)

from docling.document_converter import DocumentConverter

ModuleNotFoundError: No module named 'fitz'

In [2]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Educacion(BaseModel):
    institucion: str
    titulo: str
    fecha_inicio: Optional[str]
    fecha_fin: Optional[str]
    detalles: Optional[List[str]]

class Experiencia(BaseModel):
    empresa: str
    ubicacion: Optional[str]
    puesto: str
    fecha_inicio: Optional[str]
    fecha_fin: Optional[str]
    responsabilidades: Optional[List[str]]

class Habilidad(BaseModel):
    nombre: str
    nivel: Optional[str]

class Idioma(BaseModel):
    idioma: str
    nivel: Optional[str]

class Curriculum(BaseModel):
    nombre_completo: str
    correo: str
    telefono: Optional[str]
    resumen: Optional[str]
    experiencia: List[Experiencia]
    educacion: Optional[List[Educacion]]
    habilidades: Optional[List[Habilidad]]
    idiomas: Optional[List[Idioma]]
    certificaciones: Optional[List[str]]
    referencias: Optional[List[str]]


In [3]:
# root_dir = config.EXTERNAL_DATA_DIR / 'cvs'
root_dir = '/Users/ju/Library/CloudStorage/OneDrive-ESPARTINAS.A/DocumentacionEspartina/INNOVACION/Desarrollos propios/Base Datos CV Capital Humano'

In [4]:
file_list = []

for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.pdf'):
            file_path = os.path.join(root, file)
            # print(file_path)
            file_list.append(file_path)

file_list = tuple(file_list)
len(file_list)

333

In [11]:
query = "Eres un prolijo y laborioso data entry del sector de recursos humanos. Tu tarea es extaer muy detalladamente toda la información relevante de los curriculum vitae recibidos en formato pdf y pasarla prolijamente a una tabla excel con el formato dado. "

In [ ]:

processed = []
id_cv = 1
excluded_extensions = tuple(['.png', '.xlsx', '.jpeg'])



# Inicializar listas para cada DataFrame
candidatos_data = []
experiencia_data = []
educacion_data = []
habilidades_data = []
certificaciones_data = []

for file_path in tqdm(file_list):
    if not file_path.endswith(excluded_extensions):
        print(f'Processing {file_path}')
        try:
            source = file_path
            converter = DocumentConverter()
            result = converter.convert(source)
            result_text = result.document.export_to_markdown()
        except Exception as e:
            print(f'Not processed: {file_path} - error: {e}')
            continue

        try:
            response = beta.chat.completions.parse(
                model="gpt-4o-mini-2024-07-18",
                messages=[
                    {
                        "role": "user",
                        "content": f"Los datos del archivo {file_path} están en formato markdown:\n{result_text}\n\nPregunta: {query}"
                    }
                ],
                temperature=0,
                max_tokens=15000,
                response_format=Curriculum,
                top_p=1
            )

            json_content = response.choices[0].message.content
            data = json.loads(json_content)

            candidato_id = id_cv
            id_cv += 1
            nombre_completo = data['nombre_completo']

            # Datos generales del candidato
            candidatos_data.append({
                'candidato_id': candidato_id,
                'nombre_completo': nombre_completo,
                'correo': data['correo'],
                'telefono': data['telefono'],
                'resumen': data['resumen'],
                'file_path': file_path
            })

            # Experiencia
            for exp in data.get('experiencia', []):
                experiencia_data.append({
                    'candidato_id': candidato_id,
                    'nombre_completo': nombre_completo,
                    'empresa': exp['empresa'],
                    'ubicacion': exp['ubicacion'],
                    'puesto': exp['puesto'],
                    'fecha_inicio': exp['fecha_inicio'],
                    'fecha_fin': exp['fecha_fin'],
                    'responsabilidades': ", ".join(exp['responsabilidades']) if exp['responsabilidades'] else None
                })

            # Educación
            for edu in data.get('educacion', []):
                educacion_data.append({
                    'candidato_id': candidato_id,
                    'nombre_completo': nombre_completo,
                    'institucion': edu['institucion'],
                    'titulo': edu['titulo'],
                    'fecha_inicio': edu['fecha_inicio'],
                    'fecha_fin': edu['fecha_fin'],
                    'detalles': ", ".join(edu['detalles']) if edu['detalles'] else None
                })

            # Habilidades
            for hab in data.get('habilidades', []):
                habilidades_data.append({
                    'candidato_id': candidato_id,
                    'nombre_completo': nombre_completo,
                    'nombre': hab['nombre'],
                    'nivel': hab['nivel']
                })

            # Certificaciones
            if data.get('certificaciones'):
                for cert in data['certificaciones']:
                    certificaciones_data.append({
                        'candidato_id': candidato_id,
                        'nombre_completo': nombre_completo,
                        'certificacion': cert
                    })

            processed.append(file_path)

        except Exception as e:
            print(f'Error processing {file_path}: {e}')

# Crear los DataFrames finales
candidatos_df = pd.DataFrame(candidatos_data)
candidatos_df['zona/area'] = candidatos_df['file_path'].str.split('/', expand=True)[10]
candidatos_df = candidatos_df[['candidato_id', 'zona/area', 'nombre_completo', 'correo', 'telefono', 'resumen', 'file_path']]


experiencia_df = pd.DataFrame(experiencia_data)
experiencia_df['anio_inicio'] = experiencia_df['fecha_inicio'].str.extract(r'(\d{4})')
experiencia_df.nombre_completo = experiencia_df.nombre_completo.str.title()
experiencia_df = experiencia_df[['candidato_id', 'nombre_completo', 'empresa', 'ubicacion', 'puesto', 'anio_inicio', 'fecha_inicio', 'fecha_fin', 'responsabilidades']]


educacion_df = pd.DataFrame(educacion_data)
educacion_df.nombre_completo = educacion_df.nombre_completo.str.title()
educacion_df['anio_inicio'] = educacion_df['fecha_inicio'].str.extract(r'(\d{4})')
educacion_df = educacion_df[['candidato_id', 'nombre_completo', 'institucion', 'titulo', 'anio_inicio', 'fecha_inicio', 'fecha_fin', 'detalles']]


habilidades_df = pd.DataFrame(habilidades_data)
habilidades_df.nombre_completo = habilidades_df.nombre_completo.str.title()


certificaciones_df = pd.DataFrame(certificaciones_data)
certificaciones_df.nombre_completo = certificaciones_df.nombre_completo.str.title()

In [56]:
with pd.ExcelWriter(f'{root_dir}/base_cv_capital_humano.xlsx') as writer:
    candidatos_df.to_excel(writer, sheet_name='Candidatos', index=False)
    experiencia_df.to_excel(writer, sheet_name='Experiencia', index=False)
    educacion_df.to_excel(writer, sheet_name='Educacion', index=False)
    habilidades_df.to_excel(writer, sheet_name='Habilidades', index=False)
    certificaciones_df.to_excel(writer, sheet_name='Certificaciones', index=False)